In [1]:
# pip install light-curve

In [11]:
import numpy as np


In [2]:
import light_curve as lc
import numpy as np

# Time values can be non-evenly separated but must be an ascending array
n = 101
t = np.linspace(0.0, 1.0, n)
perfect_m = 1e3 * t + 1e2
err = np.sqrt(perfect_m)
m = perfect_m + np.random.normal(0, err)

# Half-amplitude of magnitude
amplitude = lc.Amplitude()
# Fraction of points beyond standard deviations from mean
beyond_std = lc.BeyondNStd(nstd=1)
# Slope, its error and reduced chi^2 of linear fit
linear_fit = lc.LinearFit()
# Feature extractor, it will evaluate all features in more efficient way
extractor = lc.Extractor(amplitude, beyond_std, linear_fit)

# Array with all 5 extracted features
result = extractor(t, m, err, sorted=True, check=False)

print('\n'.join(f"{name} = {value:.2f}" for name, value in zip(extractor.names, result)))

# Run in parallel for multiple light curves:
results = amplitude.many(
    [(t[:i], m[:i], err[:i]) for i in range(n // 2, n)],
    n_jobs=-1,
    sorted=True,
    check=False,
)
print("Amplitude of amplitude is {:.2f}".format(np.ptp(results)))

amplitude = 531.70
beyond_1_std = 0.40
linear_fit_slope = 1005.49
linear_fit_slope_sigma = 7.25
linear_fit_reduced_chi2 = 1.23
Amplitude of amplitude is 258.41


# [Importing DR data]

In [14]:
import pandas as pd
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture

In [15]:
file_list = glob.glob('/media3/CRP8/TDE/data/DR_photometry/extragalactic/*.json')
file_list.sort()

In [16]:
# all_lcs = []
# for file in file_list:
#     with open(file, "r") as f:
#         data = json.load(f)
        
#     #which key should I select? -- 
#     #Emille: Different keys are different matches for the same ZTFID
#     for j in range(len(data.keys())):

#         obj_id = list(data.keys())[j]
#         ztf_id = file.split('/')[-1].split('.')[0]
#         print(list(data.keys()))
#         lc = data[obj_id]["lc"]
        
#         band = data['obj_id']['meta']['filter']
        
#         mjd = [point["mjd"] for point in lc]
#         mag = [point["mag"] for point in lc]
#         magerr = [point["magerr"] for point in lc]
#         #this values have to be concatenated for under the same ZTF_ID

#         # save the light curve into a single dataframe with an unique Id (ztf_id)
#         # make the band mapping where zr,zi,zg are the bands r, i, g 
#         # and these bands/filters #1-g #2-r #3-i
    
#     # # Plot
#     # plt.figure(figsize=(8,5))
#     # plt.errorbar(mjd, mag, yerr=magerr, fmt="o", markersize=4, alpha=0.6)
#     # plt.gca().invert_yaxis()  
#     # plt.xlabel("MJD")
#     # plt.ylabel("Magnitude")
#     # plt.title(f"Light Curve: {obj_id}")
#     # plt.show()


In [17]:
file_list[0]

'/media3/CRP8/TDE/data/DR_photometry/extragalactic/ZTF17aaajnki.json'

In [18]:
with open(file_list[0], "r") as f:
        data = json.load(f)

In [26]:
for file in file_list:
    with open(file, "r") as f:
        data = json.load(f)
    
    ztf_id = file.split("/")[-1].split(".")[0]
    coords_obj = []
    field_id_list=[]
    
    for obj_id in data.keys():
        coord = data[obj_id]['meta']['coord']
        ra = coord["ra"]
        dec = coord["dec"]
        field_id = data[obj_id]['meta']['fieldid']
        coords_obj.append([ra, dec])
        field_id_list.append(field_id)
    
    X = np.array(coords_obj)
    #just in case we only have one match
    if len(X) < 2:
        print(f"Skipping {ztf_id}: not enough coords for GMM")
        continue
    
    #n_comp = min(2, len(X))
    n_comp = 2
    gmm = GaussianMixture(n_components=n_comp, random_state=0)
    gmm.fit(X)
     #Initialize KMeans with 2 clusters
    kmeans = KMeans(n_clusters=2, random_state=0, n_init=10) # n_init for multiple runs with different centroids
    
    # Fit the model to the data
    kmeans.fit(X)
    
    # Get cluster labels for each data point
    labels = kmeans.labels_
    print("Cluster Labels:", labels)
    
    # Get the coordinates of the cluster centers
    centroids = kmeans.cluster_centers_
    print("Cluster Centroids:", centroids)
    
    labels = gmm.predict(X)
    #prob = gmm.
    
    print(f"\nZTF ID: {ztf_id}")
    for obj_id, label, (ra, dec) in zip(data.keys(), labels, X):
        print(f"  ObjID: {obj_id}, RA={ra:.6f}, Dec={dec:.6f}, Cluster={label}, field_id: {np.unique(field_id_list)}")


ZTF ID: ZTF17aaajnki
  ObjID: 1714103300002371, RA=186.453552, Dec=33.546879, Cluster=0, field_id: [ 672 1714]
  ObjID: 1714203300013954, RA=186.453568, Dec=33.546879, Cluster=0, field_id: [ 672 1714]
  ObjID: 672111300009091, RA=186.453568, Dec=33.546898, Cluster=0, field_id: [ 672 1714]
  ObjID: 672211300016008, RA=186.453568, Dec=33.546894, Cluster=0, field_id: [ 672 1714]
  ObjID: 672311300009162, RA=186.453598, Dec=33.546890, Cluster=0, field_id: [ 672 1714]

ZTF ID: ZTF17aaaznul
  ObjID: 695111300026353, RA=10.084607, Dec=40.732880, Cluster=0, field_id: [695]
  ObjID: 695211300015165, RA=10.084602, Dec=40.732883, Cluster=0, field_id: [695]
  ObjID: 695311300056357, RA=10.084604, Dec=40.732872, Cluster=0, field_id: [695]

ZTF ID: ZTF17aaazxvs
  ObjID: 695111300013867, RA=9.890517, Dec=40.520737, Cluster=0, field_id: [ 695 1735]
  ObjID: 695211300021610, RA=9.890531, Dec=40.520737, Cluster=0, field_id: [ 695 1735]
  ObjID: 695311300098690, RA=9.890547, Dec=40.520756, Cluster=0, fi

In [ ]:
# import ztffields as ztf
# ids = ztf.get_fieldid(grid="main")                     # pick main or secondary grid
# fig = ztf.skyplot_fields(ids, label="fieldid")         # full-sky plot, labels = field IDs
# # Tip: for legibility, restrict to a dec/RA range:
# # ids = ztf.get_fieldid(grid="main", dec_range=[[-30, 90]])
# # fig = ztf.skyplot_fields(ids, origin=180, label="fieldid")

In [13]:
coords_obj

[dict_values([186.45355224609375, 33.546878814697266]),
 dict_values([186.4535675048828, 33.546878814697266]),
 dict_values([186.4535675048828, 33.546897888183594]),
 dict_values([186.4535675048828, 33.54689407348633]),
 dict_values([186.45359802246094, 33.54689025878906])]

Linear color coefficient term from photometric calibration
 that can be applied to the mag estimate if you know the
 AB g - r color of your source. See Cautionary Note #12 in Section 10

In [26]:
data['672311300009162']['meta'], data['672211300016008']['meta'], data['1714103300002371']['meta'], data['1714103300002371']['meta']
#1-g
#2-r
#3-i

({'nobs': 212,
  'ngoodobs': 199,
  'duration': 1784.0531100000007,
  'filter': 'zi',
  'fieldid': 672,
  'rcid': 42,
  'coord': {'ra': 186.45359802246094, 'dec': 33.54689025878906},
  'h3': {'10': 622749347444883455}},
 {'nobs': 813,
  'ngoodobs': 726,
  'duration': 2025.2762999999977,
  'filter': 'zr',
  'fieldid': 672,
  'rcid': 42,
  'coord': {'ra': 186.4535675048828, 'dec': 33.54689407348633},
  'h3': {'10': 622749347444883455}},
 {'nobs': 26,
  'ngoodobs': 22,
  'duration': 1839.911119999997,
  'filter': 'zg',
  'fieldid': 1714,
  'rcid': 10,
  'coord': {'ra': 186.45355224609375, 'dec': 33.546878814697266},
  'h3': {'10': 622749347444752383}})

In [24]:
data['672111300009091']['lc']

[{'mjd': 58202.29797,
  'mag': 16.556772,
  'magerr': 0.016116465,
  'clrcoeff': -0.025106471},
 {'mjd': 58202.34143,
  'mag': 16.526466,
  'magerr': 0.015993694,
  'clrcoeff': -0.0056083873},
 {'mjd': 58205.30794,
  'mag': 16.479694,
  'magerr': 0.015813114,
  'clrcoeff': -0.032051623},
 {'mjd': 58205.32528,
  'mag': 16.476719,
  'magerr': 0.015801983,
  'clrcoeff': -0.023014545},
 {'mjd': 58216.27002,
  'mag': 16.26345,
  'magerr': 0.015106959,
  'clrcoeff': -0.021929054},
 {'mjd': 58227.30296,
  'mag': 16.484835,
  'magerr': 0.015832445,
  'clrcoeff': -0.034358524},
 {'mjd': 58227.30391,
  'mag': 16.470161,
  'magerr': 0.015777599,
  'clrcoeff': -0.022196174},
 {'mjd': 58229.27777,
  'mag': 16.435678,
  'magerr': 0.015652688,
  'clrcoeff': -0.0982556},
 {'mjd': 58230.34126,
  'mag': 16.577751,
  'magerr': 0.016204193,
  'clrcoeff': -0.07196694},
 {'mjd': 58230.34221,
  'mag': 16.5849,
  'magerr': 0.016234605,
  'clrcoeff': -0.07149261},
 {'mjd': 58234.2465,
  'mag': 16.432753,
  'ma